### Importing necessary modules


In [1]:
import time
import matplotlib.pyplot as plt
from diffusion import (
    SequentialDiffusionEquation,
    OMPdiffusionEquation,
    CUDADiffusionEquation,
    BaseDiffusionEquation,
)
import pandas as pd
import numpy as np

### Utils Methods


In [2]:
def standard_deviation(arr: list) -> float:
    mean = sum(arr) / len(arr)
    return (sum((x - mean) ** 2 for x in arr) / len(arr)) ** 0.5

In [3]:
def measure_execution_time(
    Solver: BaseDiffusionEquation,
    N: int,
    total_eval: int,
    steps: int,
    n_threads: int = None,
) -> list:
    time_list = []
    for _ in range(total_eval):
        with Solver("../build/libDiffusionEquation.so", N=N) as solver:
            if n_threads:
                solver.set_num_threads(n_threads)

            start = time.time()
            for _ in range(steps):
                solver.step()
            end = time.time()

            time_list.append(end - start)
    return time_list

### Measure Execution time of the Different Solutions

#### Sequential implementation


In [ ]:
times = measure_execution_time(SequentialDiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

Time elapsed:  4.542544984817505 +/- 0.4173835707258996


#### OpenMP implementation


In [ ]:
times = measure_execution_time(OMPdiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

Time elapsed:  3.183614659309387 +/- 0.359222767353766


#### CUDA implementation

In [6]:
times = measure_execution_time(CUDADiffusionEquation, 500, 10, 1000)
print("Time elapsed: ", sum(times) / len(times), "+/-", standard_deviation(times))

Time elapsed:  0.9909062385559082 +/- 0.0481031679515818


### Firsts Results


Now that we can measure the execution time of both implementations, let's compare them and check if the OpenMP implementation is faster than the Sequential implementation


In [9]:
# Configs
N = 2000
total_evaluations = 15
total_steps = 500
num_threads_omp = [4]

In [ ]:
time_list = []
standard_deviation_list = []

times = measure_execution_time(SequentialDiffusionEquation, N, total_evaluations, total_steps)
time_list.append(sum(times) / len(times))
standard_deviation_list.append(standard_deviation(times))

for num_threads in num_threads_omp:
    times = measure_execution_time(OMPdiffusionEquation, N, total_evaluations, total_steps, num_threads)
    time_list.append(sum(times) / len(times))
    standard_deviation_list.append(standard_deviation(times))

times = measure_execution_time(CUDADiffusionEquation, N, total_evaluations, total_steps)
time_list.append(sum(times) / len(times))
standard_deviation_list.append(standard_deviation(times))

In [18]:
sequential_time = time_list[0]
omp_time = time_list[1]
cuda_time = time_list[2]

table = {
    "Method": ["Sequential", "OMP", "CUDA"],
    "Time (s)": [sequential_time, omp_time, cuda_time],
    "Standard Deviation": standard_deviation_list,
}

df = pd.DataFrame(table)
print(df)

df.to_csv("results.csv", index=False)

       Method   Time (s)  Standard Deviation
0  Sequential  27.204012            0.259928
1         OMP  13.376458            0.751484
2        CUDA   5.245920            0.063942


In [ ]:
sequential_time = time_list[0]

table = {
    "Num Threads": [1] + num_threads_omp,
    "Time": time_list,
    "STD": standard_deviation_list,
    "Speedup": [sequential_time / x for x in time_list],
    "Efficiency": [
        sequential_time / x / num_threads
        for x, num_threads in zip(time_list, [1] + num_threads_omp)
    ],
}

df = pd.DataFrame(table)
print(df)

df.to_csv("../data/performance/OpenMP.csv", index=False)

#### Plot the results


In [ ]:
# plot the results together in a single graph
df = pd.read_csv("../data/performance/OpenMP.csv")

plt.plot(df["Num Threads"], df["Speedup"], label="Speedup", marker="o")
# Add the linear speedup line
plt.plot(
    df["Num Threads"],
    df["Num Threads"],
    label="Speedup Linear",
    linestyle="--",
    marker="o",
)

# # Add value labels next to each data point
# for x, y in zip(df["Num Threads"], df["Speedup"]):
#     plt.text(x, y, f'{y:.2f}', fontsize=9, ha='right', va='bottom')

plt.ylim(1, df["Speedup"].max() + 0.5)  # Adjust the y-axis limit
plt.grid()
plt.title("Speedup vs Nº Threads")
plt.ylabel("Speedup")
plt.xlabel("Nº Threads")
plt.legend()
plt.show()

In [ ]:
# plot the results together in a single graph
df = pd.read_csv("../data/performance/OpenMP.csv")

# Plot the efficiency
plt.plot(df["Num Threads"], df["Efficiency"], label="Eficiência", marker="o")
# Add the linear efficiency line
plt.plot(
    df["Num Threads"],
    [1] * len(df["Num Threads"]),
    label="Eficiência Linear",
    linestyle="--",
    marker="o",
)

plt.grid()
plt.title("Eficiência vs Nº Threads")
plt.xlabel("Nº Threads")
plt.ylabel("Eficiência")
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv("../data/performance/OpenMP.csv")

# Calculate percentage of linear speedup achieved
df["Percent of Linear Speedup"] = (df["Speedup"] / df["Num Threads"]) * 100

# Plot the percentage
plt.plot(
    df["Num Threads"],
    df["Percent of Linear Speedup"],
    label="Percent of Linear Speedup",
    marker="o",
)

plt.grid()
plt.title("Percentage of Linear Speedup Achieved vs Number of Threads")
plt.xlabel("Number of Threads")
plt.ylabel("Percentage of Linear Speedup Achieved (%)")
plt.legend()
plt.show()

In [ ]:
df = pd.read_csv("../data/performance/OpenMP.csv")

fig, ax1 = plt.subplots()

color = "tab:blue"
ax1.set_xlabel("Number of Threads")
ax1.set_ylabel("Measured Speedup", color=color)
ax1.plot(
    df["Num Threads"], df["Speedup"], label="Measured Speedup", color=color, marker="o"
)
ax1.tick_params(axis="y", labelcolor=color)

ax2 = ax1.twinx()  # Instantiate a second axes sharing the same x-axis

color = "tab:red"
ax2.set_ylabel("Linear Speedup", color=color)
ax2.plot(
    df["Num Threads"],
    df["Num Threads"],
    label="Linear Speedup",
    linestyle="--",
    color=color,
)
ax2.tick_params(axis="y", labelcolor=color)

# Combined legend
lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
fig.legend(lines, labels, loc="upper left")

plt.title("Speedup vs Number of Threads")
plt.grid()
plt.show()